# Задачи 
## 1. Посчитать показатели Retention Rate седьмого дня за два периода: до и после внедрения фичи.
## 2. Посчитать показатели ARPPU за два периода: до и после внедрения фичи.

Установка необходимых библиотек

In [ ]:
#! pip install psycopg2 

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

Подсчитаем количество зарегистрировавшихся пользователей registered_users_count_may за май 2019 года в разрезе registration_date — дней регистрации.

In [ ]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count_may = pd.DataFrame(getRegisteredUsersCount())

Подсчитаем количество зарегистрировавшихся пользователей registered_users_count_june за июнь 2019 года в разрезе registration_date — дней регистрации.

In [ ]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count_june = pd.DataFrame(getRegisteredUsersCount())

In [ ]:
registered_users_count_may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
registered_users_count    31 non-null int64
registration_date         31 non-null object
dtypes: int64(1), object(1)
memory usage: 576.0+ bytes


In [ ]:
#Переводим столбец дат в соответствующий тип
registered_users_count_may['registration_date'] = pd.to_datetime(registered_users_count_may['registration_date'])
registered_users_count_june['registration_date'] = pd.to_datetime(registered_users_count_june['registration_date'])

Подсчитаем по дням количество активных пользователей active_users_count_may в мае с указанием даты регистрации registration_date.

In [ ]:
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts_may = pd.DataFrame(getActiveUsersCountWithCohorts())

Подсчитаем по дням количество активных пользователей active_users_count_june в июне с указанием даты регистрации registration_date.

In [ ]:
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts_june = pd.DataFrame(getActiveUsersCountWithCohorts())

In [ ]:
active_users_count_with_cohorts_june.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 3 columns):
active_users_count    930 non-null int64
activity_date         930 non-null object
registration_date     930 non-null object
dtypes: int64(1), object(2)
memory usage: 21.9+ KB


In [ ]:
#Переводим столбцы дат в соответствующий тип
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts_may[col] = pd.to_datetime(active_users_count_with_cohorts_may[col])
    
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts_june[col] = pd.to_datetime(active_users_count_with_cohorts_june[col])

In [ ]:
#Объединяем таблицы по колонке registration_date
retention_table_may = active_users_count_with_cohorts_may.merge(registered_users_count_may,on=['registration_date'],how='left')
retention_table_june = active_users_count_with_cohorts_june.merge(registered_users_count_june,on=['registration_date'],how='left')

## Retention Rate

In [ ]:
#Какое количество дней прошло с момента регистрации до определённого дня активности
retention_table_may['lifetime'] = retention_table_may['activity_date'] - retention_table_may['registration_date']
#Нужно получить целое количество дней, поэтому значения в колонке с типом timedelta мы разделим на константу np.timedelta64(1,'D'):
retention_table_may['lifetime'] = retention_table_may['lifetime']/np.timedelta64(1,'D')
retention_table_may['lifetime'] = retention_table_may['lifetime'].astype(int) # Приведем тип к целому числу

In [ ]:
#Retention Rate на седьмой день в мае
retention_7_table_may = retention_table_may[retention_table_may['lifetime'] == 7]
retention_7_may = retention_7_table_may.active_users_count.sum()/retention_7_table_may.registered_users_count.sum()
retention_7_may

0.12933000957960866

In [ ]:
#Какое количество дней прошло с момента регистрации до определённого дня активности
retention_table_june['lifetime'] = retention_table_june['activity_date'] - retention_table_june['registration_date']
#Нужно получить целое количество дней, поэтому значения в колонке с типом timedelta мы разделим на константу np.timedelta64(1,'D'):
retention_table_june['lifetime'] = retention_table_june['lifetime']/np.timedelta64(1,'D')
retention_table_june['lifetime'] = retention_table_june['lifetime'].astype(int) # Приведем тип к целому числу

In [ ]:
#Retention Rate на седьмой день в июне
retention_7_table_june = retention_table_june[retention_table_june['lifetime'] == 7]
retention_7_june = retention_7_table_june.active_users_count.sum()/retention_7_table_june.registered_users_count.sum()
retention_7_june

0.12560867530346634

## ARPPU

Напишем SQL-запрос, который позволит получить выручку по дням в разрезе когорт для пользователей, зарегистрировавшихся в мае 2019 года. 

In [ ]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue_may = pd.DataFrame(getRevenue())

Напишем SQL-запрос, который позволит получить выручку по дням в разрезе когорт для пользователей, зарегистрировавшихся в июне 2019 года. 

In [ ]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue_june = pd.DataFrame(getRevenue())

In [ ]:
#Переводим столбец дат в соответствующий тип
for col in ['date','registration_date']:
    revenue_may[col] = pd.to_datetime(revenue_may[col])
    
for col in ['date','registration_date']:
    revenue_june[col] = pd.to_datetime(revenue_june[col])

In [ ]:
#Переименуем столбцы дат перед объединением
revenue_may = revenue_may.rename(columns={'date':'activity_date'})
revenue_june = revenue_june.rename(columns={'date':'activity_date'})

In [ ]:
#Объединяем таблицы
retention_table_with_revenue_may = retention_table_may.merge(revenue_may,on=['registration_date','activity_date'],how='left')
retention_table_with_revenue_june = retention_table_june.merge(revenue_june,on=['registration_date','activity_date'],how='left')

In [ ]:
retention_table_with_revenue_may.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 961 entries, 0 to 960
Data columns (total 7 columns):
active_users_count          961 non-null int64
activity_date               961 non-null datetime64[ns]
registration_date           961 non-null datetime64[ns]
registered_users_count      961 non-null int64
lifetime                    961 non-null int32
revenue                     912 non-null float64
users_count_with_revenue    912 non-null float64
dtypes: datetime64[ns](2), float64(2), int32(1), int64(2)
memory usage: 56.3 KB


После объединения видим, что есть пропущенные значения. Это говорит о том, что не во все дни была выручка с пользователей.

Произведём замену пропущенных значений на 0 с помощью функции fillna().

In [ ]:
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue_may[col] = retention_table_with_revenue_may[col].fillna(0)
    
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue_june[col] = retention_table_with_revenue_june[col].fillna(0)

In [ ]:
#Преобразуемы столбцы с количеством платящих пользователей в целочисленный формат
retention_table_with_revenue_may['users_count_with_revenue'] = retention_table_with_revenue_may['users_count_with_revenue'].astype(int)
retention_table_with_revenue_june['users_count_with_revenue'] = retention_table_with_revenue_june['users_count_with_revenue'].astype(int)

In [ ]:
#ARPPU в мае
arppu_may = retention_table_with_revenue_may['revenue'].sum()/retention_table_with_revenue_may['users_count_with_revenue'].sum()
arppu_may

0.30612457926102504

In [ ]:
#ARPPU в июне
arppu_june = retention_table_with_revenue_june['revenue'].sum()/retention_table_with_revenue_june['users_count_with_revenue'].sum()
arppu_june

0.39319401107750307

# Выводы

In [ ]:
print('Retention Rate седьмого дня до внедрения фичи: {:.2%}'.format(retention_7_may))
print('Retention Rate седьмого дня после внедрения фичи: {:.2%}'.format(retention_7_june))
print('ARPPU до внедрения фичи: {:.1} у.е.'.format(arppu_may))
print('ARPPU после внедрения фичи: {:.1} у.е.'.format(arppu_june))

Retention Rate седьмого дня до внедрения фичи: 12.93%
Retention Rate седьмого дня после внедрения фичи: 12.56%
ARPPU до внедрения фичи: 0.3 у.е.
ARPPU после внедрения фичи: 0.4 у.е.


Retention Rate до внедрения фичи чуть больше, чем после. Это говорит о том, что возвращаемость и удержание пользователей на 7 день стала чуть меньше.

ARPPU выросла на 30% после внедрения фичи и теперь составляет 0.4 у.е. 

Нельзя сделать точные выводы по двум метрикам, Retention Rate остался примерно одинаковым, а на величину ARPPU погли повлиять другие факторы. Но серьезных уменьшений значений метрик не выявлено, поэтому фичу можно оставлять